# Stock movement prediction


In [1]:
# import library
import numpy as np
import pandas as pd

## data preprocess

In [2]:
# read data
df = pd.read_csv('stock.csv')
df.Date = pd.to_datetime(df.Date)
df.head()

,Date,Open Price,Close Price,High Price,Low Price,Volume
0,2009-01-02,902.99,931.80,934.73,899.35,4048270080
1,2009-01-05,929.17,927.45,936.63,919.53,5413910016
2,2009-01-06,931.17,934.70,943.85,927.28,5392620032
3,2009-01-07,927.45,906.65,927.45,902.37,4704940032
4,2009-01-08,905.73,909.73,910.00,896.81,4991549952


### new attribute

In [13]:
# 1 month line
"""
for year in range(2009, 2019):
    df2 = df[df.Date.dt.year==year]
    for month in range(1,13):
        mean = df2[df2.Date.dt.month == month].Volume.mean()
        df['One month'] = df.Volume - mean
"""     
# 
one_month = []
df2 = df.copy().set_index('Date')
for date in df2.index:
    mean = df2.loc[date - pd.Timedelta('30D'): date]
    mean = mean.Volume.mean()
    
    one_month.append(df2.loc[date].Volume - mean)
df['One month'] = one_month

df.head()

,Date,Open Price,Close Price,High Price,Low Price,Volume,One month,Rise
0,2009-01-02,902.99,931.80,934.73,899.35,4048270080,0.000000e+00,0
1,2009-01-05,929.17,927.45,936.63,919.53,5413910016,6.828200e+08,0
2,2009-01-06,931.17,934.70,943.85,927.28,5392620032,4.410200e+08,1
3,2009-01-07,927.45,906.65,927.45,902.37,4704940032,-1.849950e+08,0
4,2009-01-08,905.73,909.73,910.00,896.81,4991549952,8.129193e+07,1


In [ ]:
# ratio 

In [7]:
# rise
rise_1 = [0]
for i in range(1, df['Close Price'].size):
    if df['Close Price'][i] - df['Close Price'][i-1] >= 0:
        rise_1.append(1)
    else: rise_1.append(0)
df['Rise'] = rise_1

### splitting testing/training data

In [8]:
x_train = df[df.Date.dt.year <= 2017].drop(['Rise', 'Date'], axis=1)
y_train = df[df.Date.dt.year <= 2017].Rise

x_test = df[df.Date.dt.year > 2017].drop(['Rise', 'Date'], axis=1)
y_test = df[df.Date.dt.year > 2017].Rise


## Logistic Regression

In [9]:
from sklearn.linear_model import LogisticRegression as LR
clf = LR(random_state=0, solver='lbfgs')
clf = clf.fit(x_train, y_train)
clf.score(x_test, y_test)

0.5436507936507936

## SVM

In [10]:
from sklearn.svm import SVC
clf = SVC(gamma='scale')
clf = clf.fit(x_train, y_train)
clf.score(x_test, y_test)

0.5357142857142857

## NN

In [12]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=0.5, hidden_layer_sizes=(5, 2), random_state=1)
print(clf)
clf = clf.fit(x_train, y_train)
clf.score(x_test, y_test)

MLPClassifier(activation='relu', alpha=0.5, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)


0.4722222222222222